# Model to predict cavities at grid-points in a water box
## Approach:
We run a 50 nanosecond MD simulation of tip3p water in a 5nm by 5nm by 5nm box, with a time step of 2fs, with frames being saved after every 500 timesteps. This gives us 50000 frames in all.

We intend to supply the position coordinates of the water-oxygen atoms in each frame as inputs and for each frame a sequence of grid-points labeled as "cavity ~ 1" and "no-cavity ~ 0" as the output.
For each frame then, the network is then expected to do a binary classification for each of the output neurons (which is a sequence of the grid points) based on the input coordinates of all the water(oxygen) molecules in that frame.

In [1]:
import numpy as np
import tensorflow as tf
import keras
import os
import MDAnalysis as mda

Using TensorFlow backend.


## Data Processing

This involved two broad steps:
1. Input data preparation
2. Label Generation

### Inputs:
We dump all the individual saved frames as separate ```.gro``` files. 
We used shell scripts to process the ```.gro``` files into ```.dat``` files. The rows containing the water-hydrogen coordinates were removed (under the assumption that location of a water molecule is given by the location of the water molecules); and changed the extension to ```.dat```.
_Note: Make sure there are six distinct columns throughout the data file, otherwise import using ```np.loadtxt``` for last three columns will fail. Here, for atoms near the end of the file, the atom index becomes large and the space delimiter between the 3rd and 4th column disappears._

In [2]:
# path where .dat files are located
org_path = '/home/vatsm/Work/jarvis/ML/waterbox/data/'
dat_files = []
# load .dat files into array dat_files
for filename in os.listdir(org_path):
    if filename.endswith(".dat"):
        fulldat = os.path.join(org_path, filename)
        dat_files.append(fulldat)
# check how many files were loaded        
print(len(dat_files))

inputlist = []
# load columns of coordinates into inputlist
for item in dat_files:
    inputlist.append((np.loadtxt(item, usecols = (3,4,5,))))
print("Done.")

50001
Done.


In [3]:
# checks to see if the load worked correctly
print(len(inputlist))
print(np.ndim(inputlist))
print(np.shape(inputlist))
print(inputlist[0])

50001
3
[[3.802 3.527 2.208]
 [2.491 3.544 0.62 ]
 [4.158 0.114 3.488]
 ...
 [1.636 3.218 2.785]
 [3.673 4.341 3.048]
 [4.199 3.542 4.079]]


### Labels:

Python Package ```MDAnalysis``` was used to manipulate trajectory data and obtain the presence of cavities at specific grid points.


In [4]:
# create mda universe object with the topology and trajectory
u = mda.Universe('run.tpr','run-whole.xtc')

In [5]:
# create labels for grid points
# label contains cavity labels over all the frames
label =[]
# loop over frames in trajectory
for ts in u.trajectory:
    # array of cavity labels for each frame
    cavity =[]
    # grid size = 5 specified in the i,j,k loops
    for i in range(5) :
        for j in range(5) :
            for k in range(5) :
                # manipulation to prepare selection
                si = str(i) + ' '
                sj = str(j) + ' '
                sk = str(k) + ' '
                gridpoint = si + sj + sk
                selection = 'point ' + gridpoint + '2.0'
                # check if any items in selection and assign label accordingly
                if (len(u.select_atoms(selection))== 0):
                    cavity.append(1)
                else:
                    cavity.append(0)
    # for each frame append cavity array into label array
    
    label.append(cavity)
print("Done.")

Done.


In [6]:
# check to see labels worked correctly
np.shape(label)

(50001, 125)

Input is now loaded into ```inputlist``` and labels into ```label```. _Note: Vectorization and reshaping are still needed at this point befor putting into the model._

In [7]:
data = np.asarray(inputlist).astype('float32')

In [8]:
labels = np.asarray(label).astype('float32')

In [9]:
data.shape

(50001, 4055, 3)

In [10]:
train_data = np.reshape(data[:40000],(-1,4055*3))
train_data = train_data / 5
train_data.shape

(40000, 12165)

In [11]:
train_labels = labels[:40000]
train_labels.shape

(40000, 125)

In [12]:
val_data = np.reshape(data[40000:45000],(-1,4055*3))
val_data = val_data / 5
val_data.shape

(5000, 12165)

In [13]:
val_labels = labels[40000:45000]
val_labels.shape

(5000, 125)

In [14]:
test_data = np.reshape(data[45000:],(-1,4055*3))
test_data = test_data / 5
test_data.shape

(5001, 12165)

In [15]:
test_labels = labels[45000:]
test_labels.shape

(5001, 125)

In [21]:
from keras import models
from keras import layers

In [22]:
network = models.Sequential()
network.add(layers.Dense(256,activation='relu',input_shape=(12165,)))
network.add(layers.Dense(256,activation='relu'))
network.add(layers.Dense(256,activation='relu'))
network.add(layers.Dense(125,activation='sigmoid'))

In [23]:
network.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
# history = network.fit(train_data, train_labels, epochs= ,batch_size=, validation_data=())
network.fit(train_data, train_labels, epochs=100 , shuffle=True, batch_size=128, validation_data=(val_data, val_labels))

Train on 40000 samples, validate on 5000 samples
Epoch 1/100
40000/40000 [==============================] - 2s 38us/step - loss: 0.2138 - acc: 0.9509 - val_loss: 0.1949 - val_acc: 0.9532
Epoch 2/100
40000/40000 [==============================] - 1s 32us/step - loss: 0.1884 - acc: 0.9546 - val_loss: 0.1966 - val_acc: 0.9532
Epoch 3/100
40000/40000 [==============================] - 1s 32us/step - loss: 0.1871 - acc: 0.9546 - val_loss: 0.1904 - val_acc: 0.9532
Epoch 4/100
40000/40000 [==============================] - 1s 32us/step - loss: 0.1865 - acc: 0.9546 - val_loss: 0.1925 - val_acc: 0.9532
Epoch 5/100
40000/40000 [==============================] - 1s 32us/step - loss: 0.1862 - acc: 0.9546 - val_loss: 0.1895 - val_acc: 0.9532
Epoch 6/100
40000/40000 [==============================] - 1s 32us/step - loss: 0.1860 - acc: 0.9546 - val_loss: 0.1940 - val_acc: 0.9532
Epoch 7/100
40000/40000 [==============================] - 1s 32us/step - loss: 0.1858 - acc: 0.9546 - val_loss: 0.1903 - v

40000/40000 [==============================] - 1s 32us/step - loss: 0.1829 - acc: 0.9546 - val_loss: 0.1917 - val_acc: 0.9532
Epoch 60/100
40000/40000 [==============================] - 1s 32us/step - loss: 0.1829 - acc: 0.9546 - val_loss: 0.1927 - val_acc: 0.9532
Epoch 61/100
40000/40000 [==============================] - 1s 32us/step - loss: 0.1828 - acc: 0.9546 - val_loss: 0.1912 - val_acc: 0.9532
Epoch 62/100
40000/40000 [==============================] - 1s 32us/step - loss: 0.1828 - acc: 0.9546 - val_loss: 0.1936 - val_acc: 0.9532
Epoch 63/100
40000/40000 [==============================] - 1s 32us/step - loss: 0.1828 - acc: 0.9546 - val_loss: 0.1919 - val_acc: 0.9532
Epoch 64/100
40000/40000 [==============================] - 1s 32us/step - loss: 0.1827 - acc: 0.9546 - val_loss: 0.1938 - val_acc: 0.9532
Epoch 65/100
40000/40000 [==============================] - 1s 32us/step - loss: 0.1827 - acc: 0.9546 - val_loss: 0.1916 - val_acc: 0.9532
Epoch 66/100
40000/40000 [==============

In [25]:
test_loss, test_acc = network.evaluate(test_data, test_labels)
print('test_acc:', test_acc)

5001/5001 [==============================] - 0s 33us/step
test_acc: 0.9543947236010181
